In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from itertools import chain
import re
import json
from tqdm.autonotebook import tqdm
import string
import collections
from textblob import TextBlob
from pathlib import Path
import collections
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import utils
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils.vis_utils import plot_model

import en_core_web_lg
import spacy
from spacy.lang.en import English
from spacy.util import minibatch, compounding
from spacy import displacy

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#define stopwords
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']

In [ ]:
def clean_text(txt):
     return re.sub('[^A-Za-z0-9.]+', ' ', str(txt).lower())

In [ ]:
def text_cleaning(text, flg_stemm = False, flg_lemm = True, lst_stopwords = None):
    '''
    Converts all text to lower case, tokenize, remove multiple spaces, stopwords, stemming, lemmatize, 
    then convert all back to string
    
    text: string - name of column containing text
    lst_stopwords: list - list of stopwords to remove
    flg_stemm: bool - whether stemming is to be applied
    flg_lemm: bool - whether lemmitisation is to be applied
    '''
    
    #clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    #tokenize (convert from string to list)
    lst_text = text.split()
    
    #remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    stopwords_list]
                
    #stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    #lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    #back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
MAX_LENGTH = 64
OVERLAP = 20
    
def shorten_sentences(sentences):
    """
    Sentences that have more than MAX_LENGTH words will be split
    into multiple sentences with overlappings.
    """
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

# OBTAIN

In [ ]:
#define paths
os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')
train_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
#create a function to get the text from the JSON file and append it to the new column in table
def read_json_pub(filename, train_path = train_path, output = 'text'):
    json_path = os.path.join(train_path, (filename + '.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
#read 
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

#review
train.head()

In [ ]:
train['cleaned_label'].nunique()

# Create Sentences & Labels

In [ ]:
import nltk

DATA = []
label_count = 0
empty_count = 0

for idx,row in tqdm(train.iterrows()):
    pub = "../input/coleridgeinitiative-show-us-the-data/train/" + row.Id + ".json"            
    f = open(pub)  
    data = json.load(f)      

    balanced = False
    
    sentences = [clean_text(sentence) for sentence in nltk.sent_tokenize(str(data))]
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
    for sentence in sentences:          
     
        a = re.search(row.cleaned_label.lower(), sentence)      
        b = re.search(row.dataset_label.lower(), sentence)
        c = re.search(row.dataset_title.lower(), sentence)
        cleaned_label = row.cleaned_label.lower()
        dataset_label = row.dataset_label.lower()
        dataset_title = row.dataset_title.lower()
        
        if  a != None:
            DATA.append((sentence, cleaned_label))
            label_count = label_count + 1
            balanced = True
        elif b != None:
            DATA.append((sentence, dataset_label))
            label_count = label_count + 1
            balanced = True
        elif c != None:
            DATA.append((sentence, dataset_title))
            label_count = label_count + 1
            balanced = True            
        else:
            if balanced:
                empty_count = empty_count + 1
                balanced = False
    
print('Text with dataset:', label_count)
print('Text without dataset:', empty_count)

In [ ]:
DATA[:5]

# CLASSIFIERS: RandomForestClassifier, SVC, MultinomialNB

# Preprocessing

* Tokenize text samples into word uni+bigrams,
* Vectorize using tf-idf encoding,
* Select only the top 50,000 features from the vector of tokens by discarding tokens that appear fewer than 2 times and using f_classif to calculate feature importance.

### Train-Test-Split

In [ ]:
#save to dataframe
train_df = pd.DataFrame(DATA)
train_df = train_df.rename({0: 'Sentence', 1: 'Label'}, axis = 1)

#review
train_df.tail(10)

In [ ]:
train_df['Label'].nunique()

In [ ]:
print(train_df['Sentence'][10])
print('\n')
print(train_df['Label'][10])

In [ ]:
print(train_df['Sentence'][888])
print('\n')
print(train_df['Label'][888])

In [ ]:
print(train_df['Sentence'][51179])
print('\n')
print(train_df['Label'][51179])

In [ ]:
from sklearn.model_selection import train_test_split

X = train_df['Sentence'].to_numpy()
y = train_df['Label'].to_numpy()

In [ ]:
#split traing data into training a validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

### Vectorize Features

In [ ]:
#https://developers.google.com/machine-learning/guides/text-classification/step-3

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 2,                       #ignore terms that have a document frequency strictly lower than the given threshold
                             dtype = 'int32', 
                             strip_accents = 'unicode',
                             analyzer = 'word',
                             token_pattern = r'\w{1,}',
                             ngram_range = (1, 4), 
                             use_idf = 1,                      #enable inverse-document-frequency reweighting
                             smooth_idf = 1,                   #weights by adding one to document frequencies
                             sublinear_tf = 1,                 #apply sublinear tf scaling
                             stop_words = stopwords_list)

#learn vocabulary from training texts and vectorize training texts
vectorizer = vectorizer.fit(list(X_train) + list(X_test))

#transform Bag-of-Words textual data to numeric
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
print(train_df['Sentence'][10])
print(X_train[10])

In [ ]:
vectorizer.get_feature_names()[:20]

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

#select top 'k' of the vectorized features, limit on the number of features. We use the top 20K features
top_k = 20000

selector = SelectKBest(f_classif, k = min(top_k, X_train.shape[1]))
selector.fit(X_train, y_train)
X_train = selector.transform(X_train).astype('float32')
X_test = selector.transform(X_test).astype('float32')

In [ ]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
print(train_df['Sentence'][10])
print(X_train[10])

### Encode Label

In [ ]:
from sklearn import preprocessing

#use the LabelEncoder to convert text labels to integers, 0, 1, 2, etc.
encoder = preprocessing.LabelEncoder()

#since we have two different data set (X_train and X_test), 
#we need to fit it on all of our data otherwise there might be some categories in the test set X_test that were not in the train set X_train 
#and we will get errors
encoder.fit(list(y_train) + list(y_test)) 
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
encoder.classes_

In [ ]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
y_train

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#create a tokenization a modelling pipeline
RandomForestClassifier_model = RandomForestClassifier(n_estimators = 100, random_state = 123) #model the text

#fit the pipeline to the training data
RandomForestClassifier_model.fit(X_train, y_train)

In [ ]:
#predict 
RandomForestClassifier_yhat_train = RandomForestClassifier_model.predict(X_train)
RandomForestClassifier_yhat_test = RandomForestClassifier_model.predict(X_test)

#train accuracy 
RandomForestClassifier_train_accuracy = accuracy_score(y_train, RandomForestClassifier_yhat_train) * 100
print('Train accuracy: {0}'.format(RandomForestClassifier_train_accuracy))

#test accuracy 
RandomForestClassifier_test_accuracy = accuracy_score(y_test, RandomForestClassifier_yhat_test) * 100
print('Test accuracy: {0}'.format(RandomForestClassifier_test_accuracy))

In [ ]:
#save
filename = 'RandomForestClassifier_model.sav'
pickle.dump(RandomForestClassifier_model, open(filename, 'wb'))

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

#get cross validation to make sure model has gotten most of the patterns from the data correctly, 
#and its not picking up too much on the noise
k_fold = KFold(n_splits = 5)
RandomForestClassifier_CV = cross_val_score(RandomForestClassifier_model, X_train, y_train, 
                                            cv = k_fold, scoring = 'accuracy', n_jobs = -1).mean()
RandomForestClassifier_CV

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

#get classification report
print('Model: RandomForestClassifier', '\n', classification_report(y_test, RandomForestClassifier_yhat_test))

In [ ]:
#summary table
summary_table = pd.DataFrame({'Model': [],
                              'Accuracy': [],
                              'CV': [],
                              'Precision': [], 'Recall': [], 'F1': []})

In [ ]:
summary_table.loc[0] = ['CLF RandomForestClassifier',
                        round(accuracy_score(y_test, RandomForestClassifier_yhat_test), 2),
                        round(RandomForestClassifier_CV, 2),
                        round(precision_score(y_test, RandomForestClassifier_yhat_test, average = 'macro'), 2), 
                        round(recall_score(y_test, RandomForestClassifier_yhat_test, average = 'macro'), 2), 
                        round(f1_score(y_test, RandomForestClassifier_yhat_test, average = 'macro'), 2)]
summary_table.head()

# SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier

#create a tokenization a modelling pipeline
SGDClassifier_model = SGDClassifier(loss = 'hinge', #use Linear SVC(SVM) model when the parameter loss is set to 'hinge'
                                                      penalty = 'l2',
                                                      alpha = 1e-3,
                                                      random_state = 123,
                                                      max_iter = 5) #model the text

#fit the pipeline to the training data
SGDClassifier_model.fit(X_train, y_train) 

In [ ]:
#predict 
SGDClassifier_yhat_train = SGDClassifier_model.predict(X_train)
SGDClassifier_yhat_test = SGDClassifier_model.predict(X_test)

#train accuracy 
SGDClassifier_train_accuracy = accuracy_score(y_train, SGDClassifier_yhat_train) * 100
print('Train accuracy: {0}'.format(SGDClassifier_train_accuracy))

#test accuracy 
SGDClassifier_test_accuracy = accuracy_score(y_test, SGDClassifier_yhat_test) * 100
print('Test accuracy: {0}'.format(SGDClassifier_test_accuracy))

In [ ]:
#save
filename = 'SGDClassifier_model.sav'
pickle.dump(SGDClassifier_model, open(filename, 'wb'))

In [ ]:
#get cross validation
SGDClassifier_CV = cross_val_score(SGDClassifier_model, X_train, y_train,
                                   cv = k_fold, scoring = 'accuracy', n_jobs=-1).mean()
SGDClassifier_CV

In [ ]:
#get classification report
print('Model: Support Vector Machine', '\n', classification_report(y_test, SGDClassifier_yhat_test))

In [ ]:
summary_table.loc[1] = ['CLF Linear Support Vector Machine',
                        round(accuracy_score(y_test, SGDClassifier_yhat_test), 2),
                        round(SGDClassifier_CV, 2),
                        round(precision_score(y_test, SGDClassifier_yhat_test, average = 'macro'), 2), 
                        round(recall_score(y_test, SGDClassifier_yhat_test, average = 'macro'), 2), 
                        round(f1_score(y_test, SGDClassifier_yhat_test, average = 'macro'), 2)]
summary_table.head()

# MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

#create a tokenization a modelling pipeline
MultinomialNB_model = MultinomialNB() #model the text

#fit the pipeline to the training data
MultinomialNB_model.fit(X_train, y_train)

In [ ]:
#predict 
MultinomialNB_yhat_train = MultinomialNB_model.predict(X_train)
MultinomialNB_yhat_test = MultinomialNB_model.predict(X_test)

#train accuracy 
MultinomialNB_train_accuracy = accuracy_score(y_train, MultinomialNB_yhat_train) * 100
print('Train accuracy: {0}'.format(MultinomialNB_train_accuracy))

#test accuracy 
MultinomialNB_test_accuracy = accuracy_score(y_test, MultinomialNB_yhat_test) * 100
print('Test accuracy: {0}'.format(MultinomialNB_test_accuracy))

In [ ]:
#save
filename = 'MultinomialNB_model.sav'
pickle.dump(MultinomialNB_model, open(filename, 'wb'))

In [ ]:
#get cross validation
MultinomialNB_CV = cross_val_score(MultinomialNB_model, X_train, y_train,
                                   cv = k_fold, scoring = 'accuracy', n_jobs=-1).mean()
MultinomialNB_CV

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

#get classification report
print('Model: MultinomialNB', '\n', classification_report(y_test, MultinomialNB_yhat_test))

In [ ]:
#update summary table
summary_table.loc[2] = ['CLF MultinomialNB',
                        round(accuracy_score(y_test, MultinomialNB_yhat_test), 2),
                        round(MultinomialNB_CV, 2),
                        round(precision_score(y_test, MultinomialNB_yhat_test, average = 'macro'), 2), 
                        round(recall_score(y_test, MultinomialNB_yhat_test, average = 'macro'), 2), 
                        round(f1_score(y_test, MultinomialNB_yhat_test, average = 'macro'), 2)]
summary_table.head()

In [ ]:
summary_table.to_csv('clf_summary_table.csv')

# PREDICTION

In [ ]:
#get text
tqdm.pandas()
train['text'] = train['Id'].progress_apply(read_json_pub)

#clean text
train['text'] = train['text'].progress_apply(clean_text)

In [ ]:
from functools import partial

#read data
sample_submission = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

#apply the function to submission data
tqdm.pandas()
sample_submission['text'] = sample_submission['Id'].progress_apply(partial(read_json_pub, train_path = test_path))

#review
sample_submission.head()

In [ ]:
literal_matching = True
RandomForestClassifier_prediction = True
SGDClassifier_prediction = True
MultinomialNB_prediction = True

id_list = []
labels_list = []

for index, row in tqdm(sample_submission.iterrows()):

    sample_text = row['text']

    row_id = row['Id']
    
    #takes only the rows where train file is identical to a test file
    temp_df = train[train['text'] == clean_text(sample_text)]
    cleaned_labels = temp_df['cleaned_label'].to_list()
    
    #literal_matching 
    if literal_matching:
        for known_label in existing_labels:
            if known_label in sample_text.lower():    
                cleaned_labels.append(clean_text(known_label))
            
        print('cleaned label:', set(cleaned_labels))   
    
    #classifier 
    if RandomForestClassifier_prediction:
        
        #extract sentences
        sentences = [clean_text(sentence) for sentence in nltk.sent_tokenize(str(sample_text))]
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
        #predict
        RandomForestClassifier_labels = RandomForestClassifier_model.predict(selector.transform(vectorizer.transform(sentences).toarray()))
    
        #get labels
        RandomForestClassifier_labels = encoder.inverse_transform(RandomForestClassifier_labels) 
        print('RandomForestClassifier_label:', set(RandomForestClassifier_labels))
        RandomForestClassifier_labels = set(RandomForestClassifier_labels)
        
    cleaned_labels += RandomForestClassifier_labels

    if MultinomialNB_prediction:
        
        #extract sentences
        sentences = [clean_text(sentence) for sentence in nltk.sent_tokenize(str(sample_text))]
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
        
        #predict
        MultinomialNB_labels = MultinomialNB_model.predict(selector.transform(vectorizer.transform(sentences).toarray()))
    
        #get labels
        MultinomialNB_labels = encoder.inverse_transform(MultinomialNB_labels) 
        print('MultinomialNB_label:', set(MultinomialNB_labels))
        MultinomialNB_labels = set(MultinomialNB_labels)
        
    cleaned_labels += MultinomialNB_labels
    
    if SGDClassifier_prediction:
        
        #extract sentences
        sentences = [clean_text(sentence) for sentence in nltk.sent_tokenize(str(sample_text))]
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
        #predict
        SGDClassifier_labels = SGDClassifier_model.predict(selector.transform(vectorizer.transform(sentences).toarray()))
    
        #get labels
        SGDClassifier_labels = encoder.inverse_transform(SGDClassifier_labels) 
        print('SGDClassifier_label:', set(SGDClassifier_labels))
        SGDClassifier_labels = set(SGDClassifier_labels)
        
    cleaned_labels += SGDClassifier_labels
    
    cleaned_labels = set(cleaned_labels)
    cleaned_labels = [clean_text(x) for x in cleaned_labels]    
    labels_list.append('|'.join(cleaned_labels))
    print('label list:', labels_list)   
    id_list.append(row_id)
    print('\n')

In [ ]:
#get dataframe
sample_submission['PredictionString'] = labels_list
sample_submission

In [ ]:
print(sample_submission['PredictionString'][0])
print('\n')
print(sample_submission['PredictionString'][1])
print('\n')
print(sample_submission['PredictionString'][2])
print('\n')
print(sample_submission['PredictionString'][3])

In [ ]:
#drop 'text'
sample_submission.drop(columns = 'text', axis = 1, inplace = True)
sample_submission

In [ ]:
#save
sample_submission.to_csv('submission.csv', index = False)

#check
submission = pd.read_csv('submission.csv')
submission